# Generate LBP features and Evaluate on GBT classifier

In [1]:
from helper import *
from skimage.feature import local_binary_pattern

In [2]:
SUBSET = '20k'
DATA_DIR = 'data/'
DATA_FILENAME = '1d_subset' + SUBSET

In [3]:
(X_train, y_train), (X_test, y_test) = \
    load_npz(DATA_DIR + DATA_FILENAME + '.npz')

((20000, 9216), dtype('uint8')) ((20000,), dtype('uint8'))
((10000, 9216), dtype('uint8')) ((10000,), dtype('uint8'))


In [4]:
# Local Binary Pattern Histogram
def lbpu_histogram(img):
    patterns = local_binary_pattern(img, 8, 1, 'uniform')
    hist, _ = np.histogram(patterns, bins=np.arange(10 + 1))
    return hist.astype('int16')

In [5]:
# Turn a <v> of size 9216 (96x96) + 2304 (48x48) into 1-D LDP
# feature vector of size 40 (10x2x2) (2x2LDP histogram)
def vec_to_feat(v):
    img = v[:9216].reshape(96, 96)
    return lbpu_histogram(img)

In [6]:
Xf_train = np.apply_along_axis(delayed(vec_to_feat), 1, X_train)
Xf_train = np.concatenate(compute([x for x in Xf_train]))
Xf_train.shape, Xf_train.dtype

((20000, 10), dtype('int16'))

In [7]:
Xf_test = np.apply_along_axis(delayed(vec_to_feat), 1, X_test)
Xf_test = np.concatenate(compute([x for x in Xf_test]))
Xf_test.shape, Xf_test.dtype

((10000, 10), dtype('int16'))

In [8]:
SUBSET_FILENAME = 'lbp_subset' + SUBSET
np.savez(DATA_DIR + SUBSET_FILENAME,
         X_train=Xf_train, y_train=y_train,
         X_test=Xf_test, y_test=y_test)

In [9]:
GBT(Xf_train, Xf_test, y_train, y_test)

n_est: Train, Test
400  : 81.5,  78.3
600  : 82.7,  78.3
800  : 83.9,  78.2
